In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_part_number = df['Part_Number'].tolist()

print('总数量：' + str(len(list_part_number)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for part_number in list_part_number:
    work.put_nowait(part_number)

def crawler():
    global df
    
    while not work.empty():
        part_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                data = {'action': 'get_paginated_part_applications',
                        'page': '1',
                        'part_number': part_number}

                resp = requests.post('https://motorad.com/wp-admin/admin-ajax.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests()).text

                resp = json.loads(resp)['applications']
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                for vehicle in resp:
                    if vehicle['make'].strip() not in dict_vehicle:
                        dict_vehicle[vehicle['make'].strip()] = {vehicle['model'].strip(): {vehicle['displacement_liters'].strip(): [int(vehicle['year'].strip())]}}
                    else:
                        if vehicle['model'].strip() not in dict_vehicle[vehicle['make'].strip()]:
                            dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()] = {vehicle['displacement_liters'].strip(): [int(vehicle['year'].strip())]}
                        else:
                            if vehicle['displacement_liters'].strip() not in dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()]:
                                dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()][vehicle['displacement_liters'].strip()] = [int(vehicle['year'].strip())]
                            else:
                                dict_vehicle[vehicle['make'].strip()][vehicle['model'].strip()][vehicle['displacement_liters'].strip()].append(int(vehicle['year'].strip()))

                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        for engine in dict_vehicle[make][model]: 
                            year_max = max(dict_vehicle[make][model][engine])
                            year_min = min(dict_vehicle[make][model][engine])
                            if year_max == year_min:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine + 'L')
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine + 'L')

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Number': part_number,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Number': part_number}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：3315

1045-176  <->  [ok] - 剩余数量：3295 - 当前时间：09:17:40
1042-185  <->  [ok] - 剩余数量：3294 - 当前时间：09:17:41
1048-126  <->  [ok] - 剩余数量：3293 - 当前时间：09:17:42
1048-190  <->  [ok] - 剩余数量：3292 - 当前时间：09:17:42
1065-82_Europe_Market  <->  [ok] - 剩余数量：3291 - 当前时间：09:17:42
1049-87_Europe_Market  <->  [ok] - 剩余数量：3290 - 当前时间：09:17:42
1051-92K_Europe_Market  <->  [ok] - 剩余数量：3289 - 当前时间：09:17:42
1069-87K_Europe_Market  <->  [ok] - 剩余数量：3288 - 当前时间：09:17:42
1067-190  <->  [ok] - 剩余数量：3287 - 当前时间：09:17:42
1049-188  <->  [ok] - 剩余数量：3286 - 当前时间：09:17:42
1070-180  <->  [ok] - 剩余数量：3285 - 当前时间：09:17:42
1046-188  <->  [ok] - 剩余数量：3284 - 当前时间：09:17:42
1050KT  <->  [ok] - 剩余数量：3283 - 当前时间：09:17:42
1068-172  <->  [ok] - 剩余数量：3282 - 当前时间：09:17:42
1047-205  <->  [ok] - 剩余数量：3281 - 当前时间：09:17:43
1069-188  <->  [ok] - 剩余数量：3280 - 当前时间：09:17:44
1064-78_Europe_Market  <->  [ok] - 剩余数量：3279 - 当前时间：09:17:45
1070-82_Europe_Market  <->  [ok] - 剩余数量：3278 - 当前时间：09:17:45
1065-180  <->  [ok] - 剩余数量：3277 - 当前时间：09:17:45


1215-172  <->  [ok] - 剩余数量：3136 - 当前时间：09:18:24
1219-194  <->  [ok] - 剩余数量：3135 - 当前时间：09:18:24
1218-216  <->  [ok] - 剩余数量：3134 - 当前时间：09:18:24
1318-212  <->  [ok] - 剩余数量：3133 - 当前时间：09:18:25
1319-212  <->  [ok] - 剩余数量：3132 - 当前时间：09:18:25
1317-185  <->  [ok] - 剩余数量：3131 - 当前时间：09:18:25
1324-180  <->  [ok] - 剩余数量：3130 - 当前时间：09:18:26
1326-000  <->  [ok] - 剩余数量：3129 - 当前时间：09:18:27
1226-192  <->  [ok] - 剩余数量：3128 - 当前时间：09:18:27
1320-000  <->  [ok] - 剩余数量：3127 - 当前时间：09:18:27
1322-225  <->  [ok] - 剩余数量：3126 - 当前时间：09:18:28
1321-221  <->  [ok] - 剩余数量：3125 - 当前时间：09:18:28
1227-105_Europe_Market  <->  [ok] - 剩余数量：3124 - 当前时间：09:18:28
1227-221  <->  [ok] - 剩余数量：3123 - 当前时间：09:18:28
1229-221  <->  [ok] - 剩余数量：3122 - 当前时间：09:18:28
1325-000  <->  [ok] - 剩余数量：3121 - 当前时间：09:18:28
1230-224  <->  [ok] - 剩余数量：3120 - 当前时间：09:18:28
1234-180  <->  [ok] - 剩余数量：3119 - 当前时间：09:18:28
1233-181  <->  [ok] - 剩余数量：3118 - 当前时间：09:18:28
1232-180  <->  [ok] - 剩余数量：3117 - 当前时间：09:18:28
200-85JK_Europe_Market  <-

2029-195  <->  [ok] - 剩余数量：2978 - 当前时间：09:19:06
203-85K_Europe_Market  <->  [ok] - 剩余数量：2977 - 当前时间：09:19:06
203-88K_Europe_Market  <->  [ok] - 剩余数量：2976 - 当前时间：09:19:07
2033-192  <->  [ok] - 剩余数量：2975 - 当前时间：09:19:07
203-91_Europe_Market  <->  [ok] - 剩余数量：2974 - 当前时间：09:19:07
2033-82K_Europe_Market  <->  [ok] - 剩余数量：2973 - 当前时间：09:19:07
2042-192  <->  [ok] - 剩余数量：2972 - 当前时间：09:19:08
206-82_Europe_Market  <->  [ok] - 剩余数量：2971 - 当前时间：09:19:08
206-180  <->  [ok] - 剩余数量：2970 - 当前时间：09:19:08
2065-160  <->  [ok] - 剩余数量：2969 - 当前时间：09:19:08
2043-195  <->  [ok] - 剩余数量：2968 - 当前时间：09:19:08
2033-180  <->  [ok] - 剩余数量：2967 - 当前时间：09:19:08
206-195  <->  [ok] - 剩余数量：2966 - 当前时间：09:19:08
206-160  <->  [ok] - 剩余数量：2965 - 当前时间：09:19:08
2040-192  <->  [ok] - 剩余数量：2964 - 当前时间：09:19:10
2065-180  <->  [ok] - 剩余数量：2963 - 当前时间：09:19:10
2040-82K_Europe_Market  <->  [ok] - 剩余数量：2962 - 当前时间：09:19:11
2065-195  <->  [ok] - 剩余数量：2961 - 当前时间：09:19:11
2040-88K_Europe_Market  <->  [ok] - 剩余数量：2960 - 当前时间：09:19:11

228-82JK_Europe_Market  <->  [ok] - 剩余数量：2820 - 当前时间：09:19:46
228-88K_Europe_Market  <->  [ok] - 剩余数量：2819 - 当前时间：09:19:46
228-91JK_Europe_Market  <->  [ok] - 剩余数量：2818 - 当前时间：09:19:46
228-195  <->  [ok] - 剩余数量：2817 - 当前时间：09:19:46
228-180  <->  [ok] - 剩余数量：2816 - 当前时间：09:19:46
228-195JV  <->  [ok] - 剩余数量：2815 - 当前时间：09:19:46
229-180  <->  [ok] - 剩余数量：2814 - 当前时间：09:19:47
229-195  <->  [ok] - 剩余数量：2813 - 当前时间：09:19:47
229-82_Europe_Market  <->  [ok] - 剩余数量：2812 - 当前时间：09:19:48
247-88K_Europe_Market  <->  [ok] - 剩余数量：2811 - 当前时间：09:19:48
230-160  <->  [ok] - 剩余数量：2810 - 当前时间：09:19:48
247-88JK_Europe_Market  <->  [ok] - 剩余数量：2809 - 当前时间：09:19:48
229-170  <->  [ok] - 剩余数量：2808 - 当前时间：09:19:48
248-160  <->  [ok] - 剩余数量：2807 - 当前时间：09:19:49
248-160JV  <->  [ok] - 剩余数量：2806 - 当前时间：09:19:49
248-174  <->  [ok] - 剩余数量：2805 - 当前时间：09:19:49
247-91K_Europe_Market  <->  [ok] - 剩余数量：2804 - 当前时间：09:19:49
248-174JV  <->  [ok] - 剩余数量：2803 - 当前时间：09:19:49
248-180JV  <->  [ok] - 剩余数量：2802 - 当前时间：09:19:49

276-89K_Europe_Market  <->  [ok] - 剩余数量：2670 - 当前时间：09:20:24
273-82_Europe_Market  <->  [ok] - 剩余数量：2669 - 当前时间：09:20:26
274-180  <->  [ok] - 剩余数量：2668 - 当前时间：09:20:26
273-88JK_Europe_Market  <->  [ok] - 剩余数量：2667 - 当前时间：09:20:26
274-174  <->  [ok] - 剩余数量：2666 - 当前时间：09:20:26
273-82JK_Europe_Market  <->  [ok] - 剩余数量：2665 - 当前时间：09:20:26
281-88K_Europe_Market  <->  [ok] - 剩余数量：2664 - 当前时间：09:20:26
281-82K_Europe_Market  <->  [ok] - 剩余数量：2663 - 当前时间：09:20:26
282-170  <->  [ok] - 剩余数量：2662 - 当前时间：09:20:27
283-82K_Europe_Market  <->  [ok] - 剩余数量：2661 - 当前时间：09:20:27
281-77K_Europe_Market  <->  [ok] - 剩余数量：2660 - 当前时间：09:20:27
283-92K_Europe_Market  <->  [ok] - 剩余数量：2659 - 当前时间：09:20:27
282-77_Europe_Market  <->  [ok] - 剩余数量：2658 - 当前时间：09:20:27
284-82K_Europe_Market  <->  [ok] - 剩余数量：2657 - 当前时间：09:20:28
276-91JK_Europe_Market  <->  [ok] - 剩余数量：2656 - 当前时间：09:20:28
277-192  <->  [ok] - 剩余数量：2655 - 当前时间：09:20:30
277-82K_Europe_Market  <->  [ok] - 剩余数量：2654 - 当前时间：09:20:30
2764KT  <->  [ok] 

...

CH1002  <->  [ok] - 剩余数量：823 - 当前时间：09:26:52
TA5985FS  <->  [ok] - 剩余数量：822 - 当前时间：09:26:52
TA6015  <->  [ok] - 剩余数量：821 - 当前时间：09:26:52
CH1001  <->  [ok] - 剩余数量：820 - 当前时间：09:26:52
CH1004  <->  [ok] - 剩余数量：819 - 当前时间：09:26:53
CH1003  <->  [ok] - 剩余数量：818 - 当前时间：09:26:53
CH1005  <->  [ok] - 剩余数量：817 - 当前时间：09:26:53
CH1006  <->  [ok] - 剩余数量：816 - 当前时间：09:26:53
TA6013  <->  [ok] - 剩余数量：815 - 当前时间：09:26:53
TA9810  <->  [ok] - 剩余数量：814 - 当前时间：09:26:54
CH1009  <->  [ok] - 剩余数量：813 - 当前时间：09:26:54
TA9810FS  <->  [ok] - 剩余数量：812 - 当前时间：09:26:55
CH1008  <->  [ok] - 剩余数量：811 - 当前时间：09:26:55
TA9920  <->  [ok] - 剩余数量：810 - 当前时间：09:26:55
TA9957  <->  [ok] - 剩余数量：809 - 当前时间：09:26:55
TA9958_Europe_Market  <->  [ok] - 剩余数量：808 - 当前时间：09:26:55
TA9959FS  <->  [ok] - 剩余数量：807 - 当前时间：09:26:55
TA9959  <->  [ok] - 剩余数量：806 - 当前时间：09:26:55
TA9958  <->  [ok] - 剩余数量：805 - 当前时间：09:26:56
CH1010  <->  [ok] - 剩余数量：804 - 当前时间：09:26:56
CH1007  <->  [ok] - 剩余数量：803 - 当前时间：09:26:56
CH1011  <->  [ok] - 剩余数量：802 - 当前时间

TA9254_Europe_Market  <->  [ok] - 剩余数量：643 - 当前时间：09:27:31
TA9248_Europe_Market  <->  [ok] - 剩余数量：642 - 当前时间：09:27:31
CH2059  <->  [ok] - 剩余数量：641 - 当前时间：09:27:31
TA9253_Europe_Market  <->  [ok] - 剩余数量：640 - 当前时间：09:27:31
CH2064  <->  [ok] - 剩余数量：639 - 当前时间：09:27:31
CH2060  <->  [ok] - 剩余数量：638 - 当前时间：09:27:31
CH2065  <->  [ok] - 剩余数量：637 - 当前时间：09:27:32
CH2061  <->  [ok] - 剩余数量：636 - 当前时间：09:27:32
CH2067  <->  [ok] - 剩余数量：635 - 当前时间：09:27:33
CH2062  <->  [ok] - 剩余数量：634 - 当前时间：09:27:33
CH2068  <->  [ok] - 剩余数量：633 - 当前时间：09:27:33
CH2066  <->  [ok] - 剩余数量：632 - 当前时间：09:27:33
CH2069  <->  [ok] - 剩余数量：631 - 当前时间：09:27:33
CH2071  <->  [ok] - 剩余数量：630 - 当前时间：09:27:33
CH2073  <->  [ok] - 剩余数量：629 - 当前时间：09:27:34
CH2070  <->  [ok] - 剩余数量：628 - 当前时间：09:27:34
CH2075  <->  [ok] - 剩余数量：627 - 当前时间：09:27:34
CH2074  <->  [ok] - 剩余数量：626 - 当前时间：09:27:34
CH2077  <->  [ok] - 剩余数量：625 - 当前时间：09:27:34
CH2090  <->  [ok] - 剩余数量：624 - 当前时间：09:27:34
CH2092  <->  [ok] - 剩余数量：623 - 当前时间：09:27:34
CH2079  <->  

CH3110  <->  [ok] - 剩余数量：460 - 当前时间：09:28:09
CH3108  <->  [ok] - 剩余数量：459 - 当前时间：09:28:09
CH3085  <->  [ok] - 剩余数量：458 - 当前时间：09:28:09
CH3135  <->  [ok] - 剩余数量：457 - 当前时间：09:28:09
CH3217  <->  [ok] - 剩余数量：456 - 当前时间：09:28:10
CH3423  <->  [ok] - 剩余数量：455 - 当前时间：09:28:10
CH3097  <->  [ok] - 剩余数量：454 - 当前时间：09:28:10
CH3914  <->  [ok] - 剩余数量：453 - 当前时间：09:28:10
CH3764  <->  [ok] - 剩余数量：452 - 当前时间：09:28:10
CH3100  <->  [ok] - 剩余数量：451 - 当前时间：09:28:10
CH4015  <->  [ok] - 剩余数量：450 - 当前时间：09:28:10
CH3101  <->  [ok] - 剩余数量：449 - 当前时间：09:28:11
CH3104  <->  [ok] - 剩余数量：448 - 当前时间：09:28:11
CH3098  <->  [ok] - 剩余数量：447 - 当前时间：09:28:11
CH3103  <->  [ok] - 剩余数量：446 - 当前时间：09:28:11
CH3102  <->  [ok] - 剩余数量：445 - 当前时间：09:28:12
CH3106  <->  [ok] - 剩余数量：444 - 当前时间：09:28:12
CH3099  <->  [ok] - 剩余数量：443 - 当前时间：09:28:12
CH3087  <->  [ok] - 剩余数量：442 - 当前时间：09:28:12
CH3089  <->  [ok] - 剩余数量：441 - 当前时间：09:28:13
CH3088  <->  [ok] - 剩余数量：440 - 当前时间：09:28:13
CH3090  <->  [ok] - 剩余数量：439 - 当前时间：09:28:13
CH3091  <-

CH5408  <->  [ok] - 剩余数量：275 - 当前时间：09:28:46
CH5357  <->  [ok] - 剩余数量：274 - 当前时间：09:28:46
CH5500  <->  [ok] - 剩余数量：273 - 当前时间：09:28:46
CH5640  <->  [ok] - 剩余数量：272 - 当前时间：09:28:47
CH5513  <->  [ok] - 剩余数量：271 - 当前时间：09:28:47
CH5642  <->  [ok] - 剩余数量：270 - 当前时间：09:28:47
CH5641  <->  [ok] - 剩余数量：269 - 当前时间：09:28:47
CH5668  <->  [ok] - 剩余数量：268 - 当前时间：09:28:47
CH5920  <->  [ok] - 剩余数量：267 - 当前时间：09:28:48
CH5677  <->  [ok] - 剩余数量：266 - 当前时间：09:28:48
CH5670  <->  [ok] - 剩余数量：265 - 当前时间：09:28:48
CH5938  <->  [ok] - 剩余数量：264 - 当前时间：09:28:48
CH5692  <->  [ok] - 剩余数量：263 - 当前时间：09:28:48
CH4818  <->  [ok] - 剩余数量：262 - 当前时间：09:28:48
CH4819  <->  [ok] - 剩余数量：261 - 当前时间：09:28:48
CH4823  <->  [ok] - 剩余数量：260 - 当前时间：09:28:49
CH4820  <->  [ok] - 剩余数量：259 - 当前时间：09:28:49
CH4831  <->  [ok] - 剩余数量：258 - 当前时间：09:28:49
CH4830  <->  [ok] - 剩余数量：257 - 当前时间：09:28:50
CH4832  <->  [ok] - 剩余数量：256 - 当前时间：09:28:50
CH6007  <->  [ok] - 剩余数量：255 - 当前时间：09:28:50
CH4834  <->  [ok] - 剩余数量：254 - 当前时间：09:28:50
CH6010  <-

CH8003  <->  [ok] - 剩余数量：90 - 当前时间：09:29:23
CH8004  <->  [ok] - 剩余数量：89 - 当前时间：09:29:24
CH8001  <->  [ok] - 剩余数量：88 - 当前时间：09:29:24
CH8006  <->  [ok] - 剩余数量：87 - 当前时间：09:29:24
CH8007  <->  [ok] - 剩余数量：86 - 当前时间：09:29:24
CH8005  <->  [ok] - 剩余数量：85 - 当前时间：09:29:25
CH8008  <->  [ok] - 剩余数量：84 - 当前时间：09:29:25
CH8009  <->  [ok] - 剩余数量：83 - 当前时间：09:29:25
CH8043  <->  [ok] - 剩余数量：82 - 当前时间：09:29:25
CH8045  <->  [ok] - 剩余数量：81 - 当前时间：09:29:26
CH8046  <->  [ok] - 剩余数量：80 - 当前时间：09:29:26
CH8044  <->  [ok] - 剩余数量：79 - 当前时间：09:29:26
CH8047  <->  [ok] - 剩余数量：78 - 当前时间：09:29:26
CH8049  <->  [ok] - 剩余数量：77 - 当前时间：09:29:26
CH8051  <->  [ok] - 剩余数量：76 - 当前时间：09:29:26
CH8050  <->  [ok] - 剩余数量：75 - 当前时间：09:29:26
CH8033  <->  [ok] - 剩余数量：74 - 当前时间：09:29:26
CH8034  <->  [ok] - 剩余数量：73 - 当前时间：09:29:27
CH8032  <->  [ok] - 剩余数量：72 - 当前时间：09:29:27
CH8038  <->  [ok] - 剩余数量：71 - 当前时间：09:29:27
CH8048  <->  [ok] - 剩余数量：70 - 当前时间：09:29:27
CH8037  <->  [ok] - 剩余数量：69 - 当前时间：09:29:28
CH8035  <->  [ok] - 剩余数量：68 - 当前